In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from random import randint
from PIL import Image,ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

c:\Users\manje\anaconda3\envs\torch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225]),
    transforms.RandomRotation(randint(1,360)),
    transforms.RandomHorizontalFlip(),

])
train_path = "./Final Dataset/train"
test_path = "./Final Dataset/test"
val_path = "./Final Dataset/val"
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False
train_data = torchvision.datasets.ImageFolder(root=train_path,transform=transforms,is_valid_file=check_image)
test_data = torchvision.datasets.ImageFolder(root=test_path,transform=transforms,is_valid_file=check_image)
val_data = torchvision.datasets.ImageFolder(root=val_path,transform=transforms,is_valid_file=check_image)
batch_size = 64
num_classes = 10
train_loader = DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(dataset=test_data,batch_size=batch_size,shuffle=False)
val_loader = DataLoader(dataset=val_data,batch_size=batch_size/2,shuffle=True)

In [8]:
import torch
import torchvision.models as models
model = models.resnet50(pretrained=True)

c:\Users\KARAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\KARAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [10]:
import torch
import torch.nn as nn

in_feats=model.fc.in_features
model.fc=nn.Linear(in_feats,num_classes)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [11]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=20, gamma=0.1)

In [9]:
from tqdm import tqdm
import numpy as np
num_epochs=100
device='cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

train_losses=[]
val_loss_min=np.Inf

for epoch in tqdm(range(num_epochs)):
    train_loss = 0.0
    valid_loss = 0.0
    model.train()
    for _,(images,labels,_) in enumerate(tqdm(train_loader,desc=f'Epoch {epoch+1}')):
        images,labels=images.to(device),labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*images.size(0)
    model.eval()
    for images,labels,_ in val_loader:
        images,labels=images.to(device),labels.to(device)
        output = model(images)
        loss = criterion(output, labels)
        valid_loss += loss.item()*images.size(0)
    
    train_loss = train_loss/len(train_loader.dataset)
    val_loss = valid_loss/len(val_loader.dataset)
    train_losses.append(train_loss)
        
    print(f'\nTraining Loss: {train_loss:.4f} \tValidation Loss: {val_loss:.4f}')
    if val_loss <= val_loss_min:
        torch.save(model.state_dict(), 'resnet50.pt')
        val_loss_min = val_loss

  0%|          | 0/2 [00:08<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 46.00 MiB. GPU 0 has a total capacty of 4.00 GiB of which 0 bytes is free. Of the allocated memory 3.33 GiB is allocated by PyTorch, and 111.69 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import torch
from sklearn.metrics import classification_report

def test(model, test_loader, criterion, device, num_classes):
    model.eval()
    class_correct = [0.0] * num_classes
    class_total = [0.0] * num_classes
    test_loss = 0.0
    correct = 0
    total = 0
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            loss = criterion(output, labels)
            test_loss += loss.item() * images.size(0)
            
            _, predicted = torch.max(output, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
            
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += (predicted[i] == labels[i]).item()
                class_total[label] += 1
    
    test_loss /= len(test_loader.dataset)
    print(f'Test Loss: {test_loss:.4f}')

    for i in range(num_classes):
        if class_total[i] > 0:
            print(f'Accuracy of class {i}: {100 * class_correct[i] / class_total[i]:.2f}%')
    
    overall_accuracy = 100 * correct / total
    print(f'Overall Accuracy: {overall_accuracy:.2f}%')
    
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=[f'Class {i}' for i in range(num_classes)]))

best_model = models.resnet50(pretrained=True)
in_feats=model.fc.in_features
model.fc=nn.Linear(in_feats,num_classes)
best_model.load_state_dict(torch.load('resnet50.pt'))
best_model.to(device)

test(best_model, test_loader, criterion, device, num_classes)